# Packages

In [1]:
import pandas as pd
import numpy as np
import os
import folium

import keras

import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.ticker as ticker
from matplotlib.ticker import PercentFormatter

pd.options.display.max_columns = None
pd.options.display.max_rows = None

C:\Users\816465\AppData\Local\conda\conda\envs\TensorFlow2\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\816465\AppData\Local\conda\conda\envs\TensorFlow2\lib\site-packages\numpy\.libs\libopenblas.IPBC74C7KURV7CB2PKT5Z5FNR3SIBV4J.gfortran-win_amd64.dll
C:\Users\816465\AppData\Local\conda\conda\envs\TensorFlow2\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\816465\AppData\Local\conda\conda\envs\TensorFlow2\lib\site-packages\numpy\.libs\libopenblas.txa6yqsd3gcqqc22geq54j2udcxdxhwn.gfortran-win_amd64.dll
  stacklevel=1)
Using TensorFlow backend.
C:\Users\816465\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\816465\

In [2]:
from lib_Map import plot_map
from lib_Dummies import Master_dummies
from lib_NNT import NNTRegressor
from lib_Tiempo import tiempo
from lib_Temperatura import temperatura
from lib_Presion import Presion
from lib_Viento import Viento

# Load data

In [3]:
Data_train = pd.read_excel("./csv/Modelo de Consumo Predictivo.xlsx", sheet_name = "Data")
print(Data_train.shape)
Data_test = pd.read_excel("./csv/Modelo de Consumo Predictivo.xlsx", sheet_name = "Eval")
print(Data_test.shape)

(440806, 10)
(148931, 9)


In [4]:
Data_train.columns

Index(['ID', 'ARTICULO', 'FECHA', 'CENTROCONSUMO', 'UBICACIONVIRTUAL',
       'ID_ORGANOGESTOR', 'DESC_PROVINCIA', 'FECHACADUCIDADEXISTENCIAS',
       'IMPORTESALIDA', 'CANTIDADSALIDA'],
      dtype='object')

In [5]:
Data_test.columns

Index(['ID', 'ARTICULO', 'FECHA', 'CENTROCONSUMO', 'UBICACIONVIRTUAL',
       'ID_ORGANOGESTOR', 'DESC_PROVINCIA', 'FECHACADUCIDADEXISTENCIAS',
       'IMPORTESALIDA'],
      dtype='object')

# Data Engineer

## Creación variables año, mes y día:

### Train:

In [6]:
Data_train['Año'] = [i.year for i in Data_train['FECHA']]
Data_train['Mes'] = [i.month for i in Data_train['FECHA']]
Data_train['Dia'] = [i.day for i in Data_train['FECHA']]

### Train:

In [7]:
Data_test['Año'] = [i.year for i in Data_test['FECHA']]
Data_test['Mes'] = [i.month for i in Data_test['FECHA']]
Data_test['Dia'] = [i.day for i in Data_test['FECHA']]

In [8]:
Data_train['Año'].unique().tolist()

[2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015]

## Conversión a datos mensuales:

In [9]:
del Data_train['ID'], Data_train['FECHA'], Data_train['FECHACADUCIDADEXISTENCIAS'], Data_train['Dia']

### Train:

In [10]:
ImporteSalidaSuma = (Data_train.groupby(('ARTICULO', 'CENTROCONSUMO', 'DESC_PROVINCIA', 'UBICACIONVIRTUAL', 'Año', 'Mes')) # Agrupar
          .IMPORTESALIDA    # Quedarse con la columna packet_length
          .apply(sum)       # Calcular su suma
          .reset_index())   # Deshacer índice jerárquico

CantidadSalidaSuma = (Data_train.groupby(('ARTICULO', 'CENTROCONSUMO', 'DESC_PROVINCIA','UBICACIONVIRTUAL', 'Año', 'Mes')) # Agrupar
          .CANTIDADSALIDA    # Quedarse con la columna packet_length
          .apply(sum)       # Calcular su suma
          .reset_index())   # Deshacer índice jerárquico

lista_cols = ['ARTICULO', 'CENTROCONSUMO', 'DESC_PROVINCIA', 'Año', 'Mes']
Data_train_mensual = pd.merge(ImporteSalidaSuma, CantidadSalidaSuma,  how='inner', on= lista_cols)
Data_train_mensual.shape

C:\Users\816465\AppData\Local\conda\conda\envs\TensorFlow2\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Interpreting tuple 'by' as a list of keys, rather than a single key. Use 'by=[...]' instead of 'by=(...)'. In the future, a tuple will always mean a single key.
  """Entry point for launching an IPython kernel.
C:\Users\816465\AppData\Local\conda\conda\envs\TensorFlow2\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Interpreting tuple 'by' as a list of keys, rather than a single key. Use 'by=[...]' instead of 'by=(...)'. In the future, a tuple will always mean a single key.
  


(70816, 9)

In [11]:
Data_train_mensual.head()

,ARTICULO,CENTROCONSUMO,DESC_PROVINCIA,UBICACIONVIRTUAL_x,Año,Mes,IMPORTESALIDA,UBICACIONVIRTUAL_y,CANTIDADSALIDA
0,-1073449806,1021,Granada,974,2012,3,1.324,974,2.0
1,-1073449806,9517,Granada,1069,2012,1,0.790,1069,2.0
2,-1073449806,9517,Granada,1069,2012,4,1.324,1069,2.0
3,-1073449806,9517,Granada,1069,2012,5,0.662,1069,1.0
4,-1073449806,9538,Granada,1276,2012,1,0.000,1276,0.0


### Test

In [12]:
del Data_test['ID'], Data_test['FECHA'], Data_test['FECHACADUCIDADEXISTENCIAS'], Data_test['Dia']


In [13]:
Data_test_mensual = (Data_test.groupby(('ARTICULO', 'CENTROCONSUMO', 'DESC_PROVINCIA', 'UBICACIONVIRTUAL', 'Año', 'Mes')) # Agrupar
          .IMPORTESALIDA    # Quedarse con la columna IMPORTESALIDA
          .apply(sum)       # Calcular su suma
          .reset_index())   # Deshacer índice jerárquico
Data_test_mensual.shape

C:\Users\816465\AppData\Local\conda\conda\envs\TensorFlow2\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Interpreting tuple 'by' as a list of keys, rather than a single key. Use 'by=[...]' instead of 'by=(...)'. In the future, a tuple will always mean a single key.
  """Entry point for launching an IPython kernel.


(23199, 7)

## Deficiencias

In [14]:
len(Data_train_mensual['ARTICULO'].unique().tolist())

20

In [15]:
len(Data_test_mensual['ARTICULO'].unique().tolist())

11

In [16]:
l=[]
for i in Data_test_mensual['ARTICULO'].unique().tolist():
    if i not in Data_train_mensual['ARTICULO'].unique().tolist():
        l.append(i)
len(l)

1

## Sort values and saving as csv

In [17]:
Data_train_mensual = Data_train_mensual.sort_values(['Año', 'Mes', 'CENTROCONSUMO' ])
print(Data_train_mensual.shape)
Data_train_mensual['UBICACIONVIRTUAL'] = Data_train_mensual['UBICACIONVIRTUAL_y'] 
del Data_train_mensual['UBICACIONVIRTUAL_y'], Data_train_mensual['UBICACIONVIRTUAL_x'] 
Data_train_mensual.drop_duplicates(keep='first',inplace=True) 
print(Data_train_mensual.shape)
Data_train_mensual.to_csv('./csv/clean.csv')

(70816, 9)
(70796, 8)


In [18]:
Data_train_mensual.head()

,ARTICULO,CENTROCONSUMO,DESC_PROVINCIA,Año,Mes,IMPORTESALIDA,CANTIDADSALIDA,UBICACIONVIRTUAL
3569,-1073438649,17405,Córdoba,2008,1,0.3528,8.0,1851
3575,-1073438649,17407,Córdoba,2008,1,1.7640,40.0,1845
3580,-1073438649,17409,Córdoba,2008,1,1.7640,40.0,1848
3589,-1073438649,17414,Córdoba,2008,1,12.9213,293.0,1525
3633,-1073438649,17415,Córdoba,2008,1,2.6019,59.0,1526


In [19]:
Data_train_mensual.head()

,ARTICULO,CENTROCONSUMO,DESC_PROVINCIA,Año,Mes,IMPORTESALIDA,CANTIDADSALIDA,UBICACIONVIRTUAL
3569,-1073438649,17405,Córdoba,2008,1,0.3528,8.0,1851
3575,-1073438649,17407,Córdoba,2008,1,1.7640,40.0,1845
3580,-1073438649,17409,Córdoba,2008,1,1.7640,40.0,1848
3589,-1073438649,17414,Córdoba,2008,1,12.9213,293.0,1525
3633,-1073438649,17415,Córdoba,2008,1,2.6019,59.0,1526


## Nuevas variables

### Horas de Sol y precipitaciones:

In [20]:
# Identfy the file location of the master dataset
I_data_path = './csv/clean.csv' 

# Define file name of output .csv file
I_out_filename = 'clean_lluvia'
I_data_tiempo = './csv/lluvia/'

pd_tiempo, Data_train_lluvia = tiempo(I_data_path, I_out_filename, I_data_tiempo)

(70796, 8)
Saving dataset to ./csv/clean_lluvia.csv


In [21]:
pd_tiempo.shape

(960, 6)

In [22]:
960/(8*11)

10.909090909090908

In [23]:
Data_train_lluvia.head()

,ARTICULO,CENTROCONSUMO,DESC_PROVINCIA,Año,Mes,IMPORTESALIDA,CANTIDADSALIDA,UBICACIONVIRTUAL,Horas_Sol,Precipitacion
0,-1073438649,17405,Córdoba,2008,1,0.3528,8.0,1851,171.5,70.0
1,-1073438649,17407,Córdoba,2008,1,1.7640,40.0,1845,171.5,70.0
2,-1073438649,17409,Córdoba,2008,1,1.7640,40.0,1848,171.5,70.0
3,-1073438649,17414,Córdoba,2008,1,12.9213,293.0,1525,171.5,70.0
4,-1073438649,17415,Córdoba,2008,1,2.6019,59.0,1526,171.5,70.0


### Temperatura

In [ ]:
# Identfy the file location of the master dataset
I_data_path = './csv/clean_lluvia.csv' 

# Define file name of output .csv file
I_out_filename = 'clean_temperatura'
I_data_tiempo = './csv/Temperatura/'

pd_temp, Data_train_temp = temperatura(I_data_path, I_out_filename, I_data_tiempo)

(70796, 10)


In [ ]:
pd_temp

In [ ]:
Data_train_temp.tail()

### Presión:

In [ ]:
# Identfy the file location of the master dataset
I_data_path = './csv/clean_temperatura.csv' 

# Define file name of output .csv file
I_out_filename = 'clean_presion'

I_data_Presion = './csv/Presion/'

pd_Presion, Data_train_Presion = Presion(I_data_path, I_out_filename, I_data_Presion)

In [ ]:
Data_train_Presion.head()

In [ ]:
pd_Presion.head()

### Viento:

In [ ]:
# Identfy the file location of the master dataset
I_data_path = './csv/clean_presion.csv' 

# Define file name of output .csv file
I_out_filename = 'clean_viento'

I_data_viento = './csv/Viento/'

pd_viento, Data_train_Viento = Viento(I_data_path, I_out_filename, I_data_viento)

In [ ]:
pd_viento.head()

### Centro de Consumo string:

In [ ]:
Data = Data_train_Viento.copy()
Data['CENTROCONSUMO'] = Data['CENTROCONSUMO'].astype(str)
Data['CENTROCONSUMO_str'] = [i[0:3] for i in Data['CENTROCONSUMO']]
Data_train_Densidad = Data

### Densidad

In [ ]:
# # Identfy the file location of the master dataset
# I_data_path = './csv/clean_master_lluvia.csv' 

# # Define file name of output .csv file
# I_out_filename = 'clean_master_temperatura'

# I_data_densidad = './csv/Densidad_poblacion/'

# Densidad_df, Data_train_densidad = densidad(I_data_path, I_out_filename, I_data_densidad)

In [ ]:
Master = Data_train_Densidad.copy()
Master.to_csv('./csv/Master.csv')

# Limpieza

In [ ]:
Master.isnull().sum()

## Imputación de valores perdidos

### Temp_Max y Temp_Min:

In [ ]:
Master.loc[Master['Temp_Min'].isnull()]

In [ ]:
index_empty = Master.loc[Master['Temp_Max'].isnull()].index.tolist()
for i in index_empty:
    A = (Master["CENTROCONSUMO"] == Master["CENTROCONSUMO"][i])
    B = (Master["Mes"] == Master["Mes"][i])
    average_Temp_Max = Master.loc[A & B]['Temp_Max'].mean()
    average_Temp_Max_Tot = Master['Temp_Max'].mean()
    average_Temp_Min = Master.loc[A & B]['Temp_Min'].mean()
    average_Temp_Min_Tot = Master['Temp_Max'].mean()
    if np.isnan(average_Temp_Max):
        Master['Temp_Max'][i] = average_Temp_Max_Tot
        Master['Temp_Min'][i] = average_Temp_Min_Tot
    else:
        Master['Temp_Max'][i] = average_Temp_Max
        Master['Temp_Min'][i] = average_Temp_Min

In [ ]:
Master.isnull().sum()

## Save as csv

In [ ]:
Master.to_csv('./csv/Master_clean.csv')

# Análisis exploratorio

## Mapas:

In [ ]:
coord_path = "./csv/listado-longitud-latitud-municipios-espana.xls"
TypeData = Data_train

### Total:

In [ ]:
m = plot_map(coord_path, TypeData, Año = 'Total')
m

### Por años:

In [ ]:
for Año in Data_train['Año'].unique().tolist():
    m = plot_map(coord_path, TypeData, Año)
m

# One hot encoding

In [ ]:
# Identfy the file location of the master dataset
I_data_path = './csv/Master_clean.csv' 

# Define file name of output .csv file
I_out_filename = 'clean_master_dummies_Big'

# Max number of dummies for each feature allowed (if ones have less that Max, those features will be removed)
Max = float('inf') #to delete this bound

# Function
pd_Master_dummies1 = Master_dummies(I_data_path, I_out_filename, Max)

In [ ]:
# Identfy the file location of the master dataset
I_data_path = './csv/Master_clean.csv' 

# Define file name of output .csv file
I_out_filename = 'clean_master_dummies_Medium'

# Max number of dummies for each feature allowed (if ones have less that Max, those features will be removed)
Max = 2000  

# Function
pd_Master_dummies2 = Master_dummies(I_data_path, I_out_filename, Max)

In [ ]:
# Identfy the file location of the master dataset
I_data_path = './csv/Master_clean.csv' 

# Define file name of output .csv file
I_out_filename = 'clean_master_dummies_shorter'

# Max number of dummies for each feature allowed (if ones have less that Max, those features will be removed)
Max = 50  

# Function
pd_Master_dummies3 = Master_dummies(I_data_path, I_out_filename, Max)

# Modelo

## Short:

In [ ]:
# Identfy the file location of the master dataset
I_data_path = './csv/clean_master_dummies_shorter.csv' 
I_Model_path = './3_4_2020/Short/128'

# Parameters
k = 4
num_epochs = 100
batch_Size = 128
Verbose = 0  #1 ó 0

# Types of Models (Possibilities: 1 to 6, you can choose how many you want e.g: [2,3,1,6])
NNTList = [1,2,3,4,5,6]  

# Function
train_data, train_targets, test_data, test_targets = NNTRegressor(I_data_path, I_Model_path, k, num_epochs, 
                                                                  batch_Size, Verbose, NNTList)

In [ ]:
# Identfy the file location of the master dataset
I_data_path = './csv/clean_master_dummies_shorter.csv' 
I_Model_path = './3_5_2020/Short/16'

# Parameters
k = 4
num_epochs = 100
batch_Size = 16
Verbose = 0  #1 ó 0

# Types of Models (Possibilities: 1 to 6, you can choose how many you want e.g: [2,3,1,6])
NNTList = [1, 2, 3, 4, 5, 6]  

# Function
train_data, train_targets, test_data, test_targets = NNTRegressor(I_data_path, I_Model_path, k, num_epochs, 
                                                                  batch_Size, Verbose, NNTList)

## Medium:

In [ ]:
# Identfy the file location of the master dataset
I_data_path = './csv/clean_master_dummies_Medium.csv' 
I_Model_path = './3_4_2020/Medium/128'

# Parameters
k = 4
num_epochs = 100
batch_Size = 128
Verbose = 0  #1 ó 0

# Types of Models (Possibilities: 1 to 6, you can choose how many you want e.g: [2,3,1,6])
NNTList = [1,2,3,4,5,6]  

# Function
train_data, train_targets, test_data, test_targets = NNTRegressor(I_data_path, I_Model_path, k, num_epochs, 
                                                                  batch_Size, Verbose, NNTList)

## Big:

In [ ]:
# Identfy the file location of the master dataset
I_data_path = './csv/clean_master_dummies_Big.csv' 
I_Model_path = './3_4_2020/Big/128'

# Parameters
k = 4
num_epochs = 100
batch_Size = 128
Verbose = 0  #1 ó 0

# Types of Models (Possibilities: 1 to 6, you can choose how many you want e.g: [2,3,1,6])
NNTList = [1,2,3,4,5,6]  

# Function
train_data, train_targets, test_data, test_targets = NNTRegressor(I_data_path, I_Model_path, k, num_epochs, 
                                                                  batch_Size, Verbose, NNTList)